In [36]:
from __future__ import division, absolute_import, print_function
import pandas as pd
import os
from collections import Counter

In [65]:
mfcc_path = 'mfcc.csv'
target_path = os.path.join('..','..','feature','mfcc')
fold_def = os.path.join('..','..','feature','segmented_noFIR')
labels_path = 'labels.csv'

In [9]:
if not os.path.isdir(target_path):
    os.mkdir(target_path)

In [17]:
data_all = pd.read_csv(mfcc_path,header=None,delimiter=';')
mask = [not each[-1][0] == '0' for each in data_all[0].str.split('_')]
data_all = data_all[mask]
labels = pd.read_csv(labels_path,delimiter=';')

In [87]:
for foldnum in range(1,4):
    
    ### Load fold definition
    
    fold_path = os.path.join(fold_def,"validation%d.txt" % foldnum)
    df = pd.read_csv(fold_path,header=None)
    df[0] = ['train_'+each for each in df[0]]
    df.columns = ['filenames']
    df.set_index('filenames',drop=True,inplace=True)
    
    ### Create fold mfcc.csv
    
    files = data_all[0]
    mask1 = [each in df.index.values for each in files]
    mask2 = [not each for each in mask1]
    data_all[mask2].to_csv(os.path.join(target_path,
                                        'fold%d.train.mfcc.csv' % foldnum),
                           sep=';',header=None, index=False)
    data_all[mask1].to_csv(os.path.join(target_path,
                                        "fold%d.dev.mfcc.csv" % foldnum),
                           sep=';',header=None, index=False)
        
    ## Create fold filenames.txt
    
    files[mask2].drop_duplicates().to_csv(os.path.join(target_path,
                            "fold%d.train.filenames.txt" % foldnum),
                sep=';',header=None, index=False)
    
    files[mask1].drop_duplicates().to_csv(os.path.join(target_path,
                            "fold%d.dev.filenames.txt" % foldnum),
                sep=';',header=None, index=False)
    
    ## Create fold label.csv
    
    temp = labels.set_index('name',drop=True).drop(files[mask1].values,axis="index")
    temp.to_csv(os.path.join(target_path,
                            "fold%d.train.labels.csv" % foldnum),
                sep=';')
    
    temp = labels.set_index('name',drop=True).drop(files[mask2].values,axis="index")
    temp.to_csv(os.path.join(target_path,
                            "fold%d.dev.labels.csv" % foldnum),
                sep=';')
    